In [25]:
import os
import io
import sys
import importlib
import pandas as pd
from lxml import etree
import requests
import json
from IPython.display import Image
import sqlalchemy as sa

htmlparser =  etree.HTMLParser()

globalData = "data"


import util
importlib.reload(util)

<module 'util' from '/Users/zacharyriethof/Desktop/Final Project DS/util.py'>

In [26]:
!curl -s -o baseball.html https://www.baseball-reference.com/leagues/majors/2021-standard-pitching.shtml

In [27]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [28]:
def buildConnectionString(source="sqlite_book"):
    scheme, dbdir, database = getsqlite_creds(source=source)
    template = '{}:///{}/{}.db'
    return template.format(scheme, dbdir, database)

# Project Overview 

- Central question: What pitch did professional baseball players use the most during the 2021 season and how effective was that kind of pitch on the teams and players that used it the most and least throughout the season.   

- How will I use my data to answer the central question? To answer my central question I will first try to find the most popular pitch of the pitcharsenal.csv by finding the most popular pitch in the pitch_name column. After I find the pitch I will find the pitchers by using the first_name and last_name columns in this dataset that use the pitch the most and use the team_name_alt to find the team that uses the pitch the most. From the data I find in the pitch arsenal dataset, I will then observe how those teams performed over the 2021 season by using many of the columns in the team standard pitching dataset. 

- Datasets 

- Pitch Arsenal Stats Leaderboard(pitcharsenal.csv): This dataset contains the types of pitches each pitcher used throughout the 2021 season and the outcomes and stats for each pitch. The column names in this dataset are last_name, first_name, player_id, team_name_alt, pitch_type, pitch_name, run_value_per_100, run_value, pitches, pitch_usage, pa, ba, slg, woba, whiff_percent, k_percent, put_away, est_ba, est_slg, est_woba, hard_hit_percent. Data format of this dataset is a csv file and the number of rows in this dataset in 2377. 

- Team Standard Pitching: I will be using the Team Standard Pitching table available on the website baseball reference which contains the overall pitching stats of every team in the MLB for the 2021 season. The column names in this dataset are Tm, #P, PAge, RA/G, W, L, W-L%, ERA, G, GS, GF, CG, tSHo, SV, IP, H, R, ER, HR, BB, IBB, SO, HBP, BK, WP, BF, ERA+, FIP, WHIP, H9, HR9, BB9, SO9, SO/W, LOB. The number of rows in this dataset is 30


- Why is this data interesting to me? As I have been a big baseball fan my whole life it has always intrigued me to find new stats about the sports I love to play and watch. In this case, I am finding from the datasets I have picked the most used pitch in the 2021 MLB season but also finding what pitches were most effective and least effective for certain players and teams throughout the season. By finding the solution to my central question I will be able to know ways in how certain pitches and teams can improve their performance by cutting out certain pitches or implementing other pitches in their repertoire.     




# Citations 


- Pitch Arsenal Stats Leaderboard: https://baseballsavant.mlb.com/leaderboard/pitch-arsenal-stats?type=pitcher&pitchType=&year=2022&position=undefined&team=&min=10&sort=1&sortDir=desc

- Team Standard Pitching: https://www.baseball-reference.com/leagues/majors/2021-standard-pitching.shtml



# Process in parsing data

- I plan to create an LoL(Lists of Lists) of my Pitch Arsenal Stats Leaderboard(pitcharsenal.csv) and then from there be able to convert that LoL into a data frame called df2. 

- I plan to web scrape the table called Team Standard Pitching on the website baseball-reference.com. I will use web scraping to get the data from each column in the Team Standard Pitching table and assign them to variables such as name_list and p_list. I will create a DoL of this table so I will have to sign each column of data to a header in the DoL which looks like this DoL[“#P”] = p_list which I will have to do for all 36 columns. 

- I will then assign these DoL with the headers and columns to a dataframe called pitchingData

# Pitch Arsenal Stats

- Creating a LoL(Lists of Dictionarys) of the dataset pitcharsenal.csv



# What Columns mean in Dataframe 

- last_name - Last name of player
- first_name - First name of player
- player_id - ID of MLB player
- team_name_alt - Name of team
- pitch_type - Type of pitch
- pitch_name - Name of pitch
- run_value_per_100 - expected number of runs per 100 pitches  
- run_value - expected number of runs 
- pitches - Pitches thrown in 2021 season for that pitch
- pitch_usage - How often was the use of a pitch
- pa - Plate appears against pitch 
- ba - Batting average against pitch
- slg - Slugging percent against pitch
- woba - Weighted on base percentise against pitch
- whiff_percent - Swinging strikes per swing against pitch
- k_percent - Strikeout percentise against pitch
- put_away - Percentise of two strike counts that lead to strikeout against pitch
- est_ba - Expected batting average against pitch
- est_slg - Expected slugging percentise against pitch
- est_woba - Expected on base percentise against pitch
- hard_hit_percent - Percent of balls hit more then 95 mph against pitch 

In [29]:
def fileLoL(path):
    """
    Creates a LoL of a dataset 
    """
    filepath = open(path, 'r') # Opens the csv file pitcharsenal.csv  
    columns = filepath.readline().strip().split(",") # # Strips the first line of the csv file which is all the headers and makes a list of them split by commas and assigned to new variable columns
    
    statsData = [] # Makes new List statsData 
    for line in filepath: # For loop through the filepath pitcharsenal.csv
        data = line.strip().split(",") # Splits the data in the csv file by commas and assins it to new variable data
        statsData.append(data) # Appends the variable data to the statsData list 
    
    return columns, statsData # Returns both the lists columns and statsData
      
    

- Assigns a filepath from my data folder to use in the function fileLoL and assigns it too 2 new variables

In [30]:
file2columns, file2Data = fileLoL(os.path.join(globalData, "pitcharsenal.csv")) # Takes the pitcharsenal.csv from my data folder to use in the funtion fileLoL and assigns it to new variables

- Makes a DataFrame from the pitcharsenal.csv file into a DataFrame called df2

In [31]:
df2 = pd.DataFrame(file2Data, columns = file2columns) # Takes the vaiables file2Data and file2columns and makes them into a DatFrame

- Renaming a couple columns in my df2 Dataframe

In [32]:
df2.rename(columns = {'\ufefflast_name': 'last_name', ' first_name': 'first_name'}, inplace = True)

- Calls the Dataframe df2

In [33]:
df2

,last_name,first_name,player_id,team_name_alt,pitch_type,pitch_name,run_value_per_100,run_value,pitches,pitch_usage,...,ba,slg,woba,whiff_percent,k_percent,put_away,est_ba,est_slg,est_woba,hard_hit_percent
0,Burnes,Corbin,669203,MIL,FC,Cutter,-1.2,-21,1815,55.4,...,0.213,0.335,0.292,27.8,24.6,24.1,0.246,0.376,0.312,38.4
1,Rodón,Carlos,607074,SF,FF,4-Seamer,-1.2,-22,1826,61.2,...,0.213,0.358,0.29,27.9,30.5,24.1,0.206,0.347,0.278,44.8
2,Gilbert,Logan,669302,SEA,FF,4-Seamer,-0.7,-12,1626,53.9,...,0.259,0.439,0.337,23.7,22.5,20.5,0.262,0.444,0.339,50.5
3,McKenzie,Triston,663474,CLE,FF,4-Seamer,-1.1,-17,1570,55.9,...,0.203,0.358,0.29,19.6,18.2,17.2,0.241,0.451,0.337,46.9
4,Cole,Gerrit,543037,NYY,FF,4-Seamer,-0.5,-8,1698,51.9,...,0.221,0.42,0.313,28.9,33.8,24.8,0.216,0.398,0.3,47.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,Yajure,Miguel,664337,PIT,FC,Cutter,11.9,4,36,7.6,...,0.6,1,0.775,28.6,20,33.3,0.352,0.477,0.356,50
2372,Muller,Kyle,666205,ATL,CUKC,Curveball,2.1,1,46,19.7,...,0.2,0.3,0.285,38.9,40,14.8,0.288,0.409,0.371,33.3
2373,Contreras,Roansy,672710,PIT,CH,Changeup,5.5,3,46,2.9,...,0.333,0.667,0.54,0,0,0,0.301,0.406,0.345,22.2
2374,Naile,James,664942,STL,CUKC,Curveball,-2.9,-1,32,25.4,...,0.2,0.2,0.18,25,20,11.8,0.288,0.339,0.28,50


# Team Standard Pitching

- Web Scraping a data table Team Standard Pitching from the website baseball-reference.com 

- Website: https://www.baseball-reference.com/leagues/majors/2021-standard-pitching.shtml 

# Why is this table on baseball-reference.com eligible for web scraping?
- On the website baseball-reference.com under the terms & conditions it says that allow anybody to access, view, or use content or material available on there website if you read and agree too the terms & services on there website.  

- https://www.sports-reference.com/termsofuse.html

- Quote = "respect to your access and use of the Site and of any and all information or data of any kind arising from access or use of the Site, including, but not limited to, all statistics, facts, text, graphics, images, forms, artwork, sound recordings, audio, video, and software available through the Site"

In [35]:
# reads HTML from the web 
resource_path = "/leagues/majors/2021-standard-pitching.shtml" # Creates resource path to my website which is baseball reference

url = util.buildURL(resource_path, "baseball-reference.com") # Builds a url using the resource_path
response = requests.get(url)
assert response.status_code == 200

# Use a custom HTML parser to parse the response content into an XML Element
tree = etree.parse(io.BytesIO(response.content), htmlparser)
root = tree.getroot()


# Testing if in correct table

In [36]:
tableroot = root.xpath("//table")[0] # Grabs the first table on my website which is the one I am using 

util.print_xml(tableroot, depth=16, nlines=20) # Prints the table I am grabbing 

<table class='sortable stats_table' id='teams_standard_pitc
  <caption>Team Standard Pitching Table</caption>
  <colgroup>
    <col></col>
    <col></col>
    <col></col>
    <col></col>
    <col></col>
    <col></col>
    <col></col>
    <col></col>
    <col></col>
    <col></col>
    <col></col>
    <col></col>
    <col></col>
    <col></col>
    <col></col>
    <col></col>
     ...


# Web Scraping table into a Dataframe

# What Columns mean in Dataframe 
- Tm - Team name 
- NP - Number of pitches used in game 
- PAge - Pitchers average age 
- RAPG - Runs allowed per game 
- W - Wins
- L - Loses
- WLP - Win to lose percentage 
- ERA - Earned run percentage
- G - Games played
- GS - Games started 
- GF - Games finished 
- CG - Complete games 
- tSho - Shutouts by a team 
- cSho - Shutouts by individual player 
- SV - Saves 
- IP - Innings pitched
- H - Hits allowed
- R - Runs allowed 
- ER - Earned runs allowed 
- HR - Home runs allowed
- BB - Bases on ball/walks 
- IBB - Intentional bases on balls/walks 
- SO - Strikeouts
- HBP - Hit by pitch
- BK - Balks
- WP - Wild pitches
- BF - Batters faced
- ERAP - Adjusted earned run average 
- FIP - Field indepedent pitching
- WHIP - Hits and walks per inning 
- H9 - Hits per 9 innings 
- HR9 - Home runs per 9 innings 
- BB9 - Bases on balls/walks per 9 innings 
- SO9 - Strikeouts per 9 innings 
- SOWR - Strikeout to walk ratio 
- LOB - Runners left on base




In [37]:
DoL = {} # Creation of DoL

# Creates a path to each nunique column in the Team Standard Pitching table
name_list = root.xpath("//tbody[position () = 1]/tr/th/a/text()")
p_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 1]/text()")
page_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 2]/text()")
RAG_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 3]/text()")
wins_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 4]/text()")
loses_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 5]/text()")
percent_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 6]/text()")
ERA_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 7]/text()")
games_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 8]/text()")
gamesp_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 9]/text()")
gamef_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 10]/text()")
CG_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 11]/text()")
tsho_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 12]/text()")
csho_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 13]/text()")
saves_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 14]/text()")
innings_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 15]/text()")
hits_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 16]/text()")
runs_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 17]/text()")
er_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 18]/text()")
hr_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 19]/text()")
bb_list =  root.xpath("//tbody[position () = 1]/tr/td[position () = 20]/text()")
ibb_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 21]/text()")
so_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 22]/text()")
hbp_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 23]/text()")
bk_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 24]/text()")
wp_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 25]/text()")
bf_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 26]/text()")
era_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 27]/text()")
fip_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 28]/text()")
whip_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 29]/text()")
h9_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 30]/text()")
hr9_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 31]/text()")
bb9_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 32]/text()")
so9_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 33]/text()")
sow_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 34]/text()")
lob_list = root.xpath("//tbody[position () = 1]/tr/td[position () = 35]/text()")

# Assigns each list of data from column to a header of each column in the DoL
DoL["Tm"] = name_list
DoL["#P"] = p_list
DoL["PAge"] = page_list
DoL["RA/G"] = RAG_list
DoL["W"] = wins_list
DoL["L"] = loses_list
DoL["W-L%"] = percent_list
DoL["ERA"] = ERA_list
DoL["G"] = games_list
DoL["GS"] = gamesp_list
DoL["GF"] = gamef_list 
DoL["CG"] = CG_list
DoL["tSho"] = tsho_list
DoL["cSho"] = csho_list
DoL["SV"] = saves_list
DoL["IP"] = innings_list
DoL["H"] = hits_list
DoL["R"] = runs_list
DoL["ER"] = er_list
DoL["HR"] = hr_list
DoL["BB"] = bb_list 
DoL["IBB"] = ibb_list
DoL["SO"] = so_list
DoL["HBP"] = hbp_list
DoL["BK"] = bk_list
DoL["WP"] = wp_list
DoL["BF"] = bf_list 
DoL["ERA+"] = era_list
DoL["FIP"] = fip_list 
DoL["WHIP"] = whip_list 
DoL["H9"] = h9_list
DoL["HR9"] = hr9_list
DoL["BB9"] = bb9_list 
DoL["SO9"] = so9_list
DoL["SO/W"] = sow_list 
DoL["LOB"] = lob_list


In [38]:
pitchingData = pd.DataFrame(DoL) # Take the variable pitchingData which has the DoL function in it and makes it a DataFrame

In [39]:
pitchingData.rename(columns = {'#P': 'NP', 'RA/G': 'RAPG', 'W-L%': 'WLP', 'ERA+': 'ERAP', 'SO/W': 'SOWR'}, inplace = True)

In [40]:
pitchingData # Calls the pitchingData DataFrame 

,Tm,NP,PAge,RAPG,W,L,WLP,ERA,G,GS,...,BF,ERAP,FIP,WHIP,H9,HR9,BB9,SO9,SOWR,LOB
0,Arizona Diamondbacks,41,28.5,5.51,52,110,.321,5.11,162,162,...,6270,83,4.88,1.436,9.4,1.5,3.5,7.9,2.23,1146
1,Atlanta Braves,30,29.5,4.07,88,73,.547,3.88,161,161,...,5944,112,4.09,1.243,7.9,1.2,3.3,9.0,2.75,1074
2,Baltimore Orioles,42,28.1,5.90,52,110,.321,5.84,162,162,...,6252,77,5.15,1.484,9.7,1.7,3.6,7.9,2.19,1103
3,Boston Red Sox,37,29.7,4.62,92,70,.568,4.26,162,162,...,6147,110,3.95,1.378,8.9,1.1,3.5,9.7,2.80,1163
4,Chicago Cubs,40,29.4,5.18,71,91,.438,4.87,162,162,...,6202,87,4.88,1.403,8.8,1.5,3.8,8.7,2.28,1143
5,Chicago White Sox,27,28.3,3.93,93,69,.574,3.73,162,162,...,5861,118,3.74,1.204,7.7,1.2,3.1,10.2,3.27,1029
6,Cincinnati Reds,36,28.9,4.69,83,79,.512,4.40,162,162,...,6199,107,4.34,1.357,8.3,1.3,3.9,9.6,2.47,1161
7,Cleveland Indians,25,26.3,4.49,80,82,.494,4.34,162,162,...,5968,99,4.43,1.281,8.2,1.4,3.3,8.9,2.66,1035
8,Colorado Rockies,26,28.3,4.94,74,87,.460,4.82,161,161,...,6044,100,4.47,1.386,9.0,1.3,3.5,8.2,2.35,1081
9,Detroit Tigers,29,27.3,4.67,77,85,.475,4.32,162,162,...,6164,99,4.60,1.367,8.7,1.3,3.6,8.0,2.20,1174


# Building the connection to my SQL database

In [41]:
fstring = buildConnectionString("sqlite_finalproject")
print("Connection string:",fstring)

engine = sa.create_engine(fstring)
connection = engine.connect()

Connection string: sqlite:///./finalproject.db


# SQL statements for tables 

 

- pitcharsenal table that involves all the columns from the pitcharsenal.csv

In [42]:
query2 = """
   CREATE TABLE IF NOT EXISTS pitcharsenal (
   last_name VARCHAR(20) NOT NULL,
   first_name VARCHAR(20) NOT NULL,
   player_id INT NOT NULL,
   team_name_alt VARCHAR(3) NOT NULL, 
   pitch_type VARCHAR(10) NOT NULL, 
   pitch_name VARCHAR(15) NOT NULL,
   run_value_per_100 FLOAT NOT NULL, 
   run_value INT NOT NULL,  
   pitches INT NOT NULL,
   pitch_usage FLOAT NOT NULL,
   pa INT NOT NULL, 
   ba FLOAT NOT NULL, 
   slg FLOAT NOT NULL,
   woba FLOAT NOT NULL, 
   whiff_percent FLOAT NOT NULL,  
   k_percent FLOAT NOT NULL,  
   put_away FLOAT NOT NULL,  
   est_ba FLOAT NOT NULL,
   est_slg FLOAT NOT NULL,
   est_woba FLOAT NOT NULL,
   hard_hit_percent VARCHAR(10) NOT NULL,
   PRIMARY KEY (pitch_name, player_id)
)
     
"""

# Execute Statement 
try:
    connection.execute(query2)
except sa.exc.SQLAlchemyError as err:
    print("CREATE of pitchtype failed:", str(err))

In [43]:
# Close the connection!
try:
    connection.close()
except:
    pass
del engine

- teamstats table that involves columns from the Team Standard Pitching table on the baseballreference website 

In [44]:
fstring = buildConnectionString("sqlite_finalproject")
print("Connection string:",fstring)

engine = sa.create_engine(fstring)
connection = engine.connect()

Connection string: sqlite:///./finalproject.db


In [45]:
query3 = """
    CREATE TABLE IF NOT EXISTS teamstats (
    Tm VARCHAR(40) NOT NULL,
    NP INT NOT NULL,
    PAge FLOAT NOT NULL,
    RAPG FLOAT NOT NULL,
    W INT NOT NULL,
    L INT NOT NULL,
    WLP FLOAT NOT NULL,
    ERA FLOAT NOT NULL,
    G INT NOT NULL,
    GS INT NOT NULL,
    GF INT NOT NULL,
    CG INT NOT NULL,
    tSho INT NOT NULL,
    cSho INT NOT NULL,
    SV INT NOT NULL,
    IP FLOAT NOT NULL,
    H INT NOT NULL,
    R INT NOT NULL,
    ER INT NOT NULL,
    HR INT NOT NULL,
    BB INT NOT NULL,
    IBB INT NOT NULL,
    SO INT NOT NULL,
    HBP INT NOT NULL,
    BK INT NOT NULL,
    WP INT NOT NULL,
    BF INT NOT NULL,
    ERAP INT NOT NULL,
    FIP FLOAT NOT NULL,
    WHIP FLOAT NOT NULL,
    H9 FLOAT NOT NULL,
    HR9 FLOAT NOT NULL,
    BB9 FLOAT NOT NULL,
    SO9 FLOAT NOT NULL,
    SOWR FLOAT NOT NULL,
    LOB INT NOT NULL,
    PRIMARY KEY (Tm)

)    
"""

# Execute Statement 
try:
    connection.execute(query3)
except sa.exc.SQLAlchemyError as err:
    print("CREATE of teamstats failed:", str(err))

In [46]:
# Close the connection!
try:
    connection.close()
except:
    pass
del engine

# SQL Statements to populate the tables

- Populating my pitcharsenal SQL table with the df2 dataframe

In [47]:
fstring = buildConnectionString("sqlite_finalproject")
print("Connection string:",fstring)

engine = sa.create_engine(fstring)
connection = engine.connect()

Connection string: sqlite:///./finalproject.db


In [48]:
df2.to_sql("pitcharsenal", con=connection, if_exists="append", index=False)

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: pitcharsenal.pitch_name, pitcharsenal.player_id
[SQL: INSERT INTO pitcharsenal (last_name, first_name, player_id, team_name_alt, pitch_type, pitch_name, run_value_per_100, run_value, pitches, pitch_usage, pa, ba, slg, woba, whiff_percent, k_percent, put_away, est_ba, est_slg, est_woba, hard_hit_percent) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (('Burnes', ' Corbin', '669203', 'MIL', 'FC', 'Cutter', '-1.2', '-21', '1815', '55.4', '464', '0.213', '0.335', '0.292', '27.8', '24.6', '24.1', '0.246', '0.376', '0.312', '38.4'), ('Rodón', ' Carlos', '607074', 'SF', 'FF', '4-Seamer', '-1.2', '-22', '1826', '61.2', '443', '0.213', '0.358', '0.29', '27.9', '30.5', '24.1', '0.206', '0.347', '0.278', '44.8'), ('Gilbert', ' Logan', '669302', 'SEA', 'FF', '4-Seamer', '-0.7', '-12', '1626', '53.9', '418', '0.259', '0.439', '0.337', '23.7', '22.5', '20.5', '0.262', '0.444', '0.339', '50.5'), (

- Populating my teamstats SQL table with the pitchingData dataframe  

In [24]:
pitchingData.to_sql("teamstats", con=connection, if_exists="append", index=False)

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: teamstats.Tm
[SQL: INSERT INTO teamstats ("Tm", "NP", "PAge", "RAPG", "W", "L", "WLP", "ERA", "G", "GS", "GF", "CG", "tSho", "cSho", "SV", "IP", "H", "R", "ER", "HR", "BB", "IBB", "SO", "HBP", "BK", "WP", "BF", "ERAP", "FIP", "WHIP", "H9", "HR9", "BB9", "SO9", "SOWR", "LOB") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (('Arizona Diamondbacks', '41', '28.5', '5.51', '52', '110', '.321', '5.11', '162', '162', '159', '3', '4', '3', '22', '1417.1', '1480', '893', '804', '232', '555', '45', '1238', '75', '7', '43', '6270', '83', '4.88', '1.436', '9.4', '1.5', '3.5', '7.9', '2.23', '1146'), ('Atlanta Braves', '30', '29.5', '4.07', '88', '73', '.547', '3.88', '161', '161', '159', '2', '18', '2', '40', '1410.2', '1237', '656', '608', '183', '516', '34', '1417', '66', '8', '73', '5944', '112', '4.09', '1.243', '7.9', '1.2', '3.3', '9.0', '2.75'

# Relationships between Tables 

- Both tables share the relationship of having a team name column as in the pitcharsenal database there is the column team_name_alt and in the teamstats the Tm column. 

- The team name columns as team_name_alt and Tm are the way I have been able to connect an compare information in both of my tables as I have been able to compare individual players in the pitcharsenal dataset to there assigned teams in the teamstats dataset. This relationship made it easy for me to make certain graphs using both tables data.  

# Databases designs and why my databases both adheres the principles of a good database

- pitcharsenal database
    - Design: The pitcharsenal database was designed to have 21 columns with 2376 rows with 2 of these columns mapping to the other 19 columns. The two primary keys in this database are player_id and pitch_name which are dependent on all the other columns of data in the database. 
    - Why its a good database:
        - All my primary keys that are independent variables player_id and pitch_name are responable for all the other columns which are all dependent variables. 
        - All my columns in my database map to a single value. 
        - No values in my database are NULL or do not have a entry
        - All data in each column are all the same data type
- teamstats database 
    - Design: The teamstats database was designed to have 36 columns with only 30 rows of data. This database will only have one primary key which is Tm column that is the team names of every team in the MLB. 
    - Why its a good database: 
        - All my primary keys that are independent variables such as Tm are responable for all the other columns that are the dependent variables. 
        - All my columns in my database map to a single value.
        - No values in my database are NULL or do not have a entry. 
        - All data in each column are all the same data type

# Challenges faced when storing data in SQL database

- When making my SQL table named teamstats I had problems with many of my columns names as SQL did not accept if my column names had any #, /, %, - in it. This is made me go back into my original dataset and rename a couple columns. 

- I originally had problems editing and making a creds.json file but I was able to get help

- For my df2 dataframe my original ideas was to have two datasets singleplayer and pitchtype due to a error when creating the singleplayer SQL table with me needing another primary key I figured out that both my tables had the same primary keys. From this I then changed too just combining both the singleplayer table and the pitchtype table into one SQL table called pitcharsenal  